In [72]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [73]:
import pyhf
import logging
logging.basicConfig(level = logging.ERROR)
reload(pyhf)
from pyhf import hfpdf
def prep_data(sourcedata):
    spec =  {
        'signal': {
            'signal': {
                'data': sourcedata['signal']['bindata']['sig'],
                'mods': [
                    {
                        'name': 'mu',
                        'type': 'normfactor',
                        'data': None
                    }
                ]
            },
            'background': {
                'data': sourcedata['signal']['bindata']['bkg'],
                'mods': [
                    {
                        'name': 'uncorr_bkguncrt_signal',
                        'type': 'shapesys',
                        'data': sourcedata['signal']['bindata']['bkgerr']
                    }
                ]
            }
        },
        'control': {
            'background': {
                'data': sourcedata['control']['bindata']['bkg'],
                'mods': [
                    {
                        'name': 'uncorr_bkguncrt_control',
                        'type': 'shapesys',
                        'data': sourcedata['control']['bindata']['bkgerr']
                    }
                ]
            }            
        }
    }
    pdf  = hfpdf(spec)
    data = []
    for c in pdf.channel_order:
        data += sourcedata[c]['bindata']['data']
    data = data + pdf.auxdata
    return data, pdf

In [74]:
source = {
  "channels": {
    "signal": {
      "binning": [2,-0.5,1.5],
      "bindata": {
        "data":    [110.0, 155.0],
        "bkgerr":  [10.0, 10.0],
        "bkg":     [100.0, 150.0],
        "sig":     [10.0, 35.0]
      }
    },
    "control": {
      "binning": [2,-0.5,1.5],
      "bindata": {
        "data":    [205.0, 345.0],
        "bkg":     [200.0, 350.0],
        "bkgerr":  [5.0, 10.0]
      }
    }
  }
}

d,pdf = prep_data(source['channels'])

print d

init_pars = [1.0,1.0,1.0,1.0,1.0]
par_bounds = [[0,10],[0,10],[0,10],[0,10],[0,10]]


print pdf.pdf(init_pars, d)

unconpars = pyhf.unconstrained_bestfit(d,pdf,init_pars,par_bounds)
print 'UNCON',unconpars


# print d
# print pdf.expected_data(unconpars)


conpars = pyhf.constrained_bestfit(0.0,d,pdf,init_pars,par_bounds)
print 'CONS', conpars


# print pdf.expected_data(conpars)

# # print '????',aux
# aux = pdf.expected_auxdata(conpars)
# # print '????',aux

# print 'ASIMOV',pyhf.generate_asimov_data(0.0,d,pdf,init_pars,par_bounds)

[205.0, 345.0, 110.0, 155.0, 1600.0, 1225.0, 100.0, 225.0]
6.444423901300363e-15
UNCON [1.00225339 0.99620861 0.21647205 1.03451146 0.99071727]


/Users/lukas/Code/histfactory/pyhf/pyhf/__init__.py:316: RuntimeWarning: divide by zero encountered in log
  for d,lam in zip(actual_data, lambdas_data):
/Users/lukas/Code/histfactory/pyhf/pyhf/__init__.py:304: RuntimeWarning: divide by zero encountered in log
  for a,alpha in zip(thisauxdata, modalphas):


CONS [1.00225277 0.99620813 0.         1.0462051  1.01080135]


In [ ]:
def plot_results(testmus,cls_obs, cls_exp, test_size = 0.05):
    plt.plot(mutests,cls_obs, c = 'k')
    for i,c in zip(range(5),['grey','grey','grey','grey','grey']):
        plt.plot(mutests,cls_exp[i], c = c)
    plt.plot(testmus,[test_size]*len(testmus), c = 'r')
    plt.ylim(0,1)

def CLsOnePoint(muTest,d,pdf, init_pars,par_bounds):
    clsobs, cls_exp = pyhf.runOnePoint(muTest, d,pdf,init_pars,par_bounds)[-2:]
    return 1./clsobs, [1./x for x in cls_exp]

def invert_interval(testmus,cls_obs, cls_exp, test_size = 0.05):
    point05cross = {'exp':[],'obs':None}
    for cls_exp_sigma in cls_exp:
        yvals = [x for x in cls_exp_sigma]
        point05cross['exp'].append(np.interp(test_size,list(reversed(yvals)),list(reversed(testmus))))
        
    yvals = cls_obs
    point05cross['obs'] = np.interp(test_size,list(reversed(yvals)),list(reversed(testmus)))
    return point05cross


CLsOnePoint(1.0,d,pdf,init_pars,par_bounds)


mutests = np.linspace(0,5,61)
tests = [CLsOnePoint(muTest,d,pdf,init_pars,par_bounds) for muTest in mutests]
cls_obs = [test[0]  for test in tests]
cls_exp = [[test[1][i]  for test in tests] for i in range(5)]

plot_results(mutests, cls_obs, cls_exp)

invert_interval(mutests, cls_obs, cls_exp)




In [ ]:
pdf.config.nuis_map